# Thinking 1 ：什么是反向传播中的链式法则

答：  
最终结果的误差是由前面几层神经网络传递而来，要想求得每一层的误差，就需要使用每一层后面一层的误差乘上导数，每一层都是如此，中间不可以中断，一直到第一层神经网络，会发现第一层神经网络权重的修正值是后面几层权重转置累乘与最终误差的乘积。

# Thinking 2 ： 请列举几种常见的激活函数，激活函数有什么作用

答：  
ReLU；Sigmoid；tanh，激活函数的作用是增加神经网络的非线性特征。

# Thinking 3 ：利用梯度下降法训练神经网络，发现模型loss不变，可能有哪些问题？怎么解决？

答：  
1. 权重未更新 -->> 使用SGD, Adam 等优化方法对去权重进行更新。
2. 梯度消失，权重每次更新的幅度太小 -->> 更换激活函数，使用 ReLU 避免发生梯度消失现象。
3. 使用梯度下降时，学习率 Learning rate 太小，每次更新参数都不明显 -->> 使用Adam 自适应更新参数。

# Action 1 :使用Pytorch编写神经网络，完成boston房价预测问题
>1）数据加载：from sklearn.datasets import load_boston  
2）网络结构：  
l1 = Linear(X, W1, b1)  
s1 = Relu(l1)  
l2 = Linear(s1, W2, b2)  
cost = MSE(y, l2)   
隐藏层维度为10  

# 一、使用 Numpy

In [44]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

#加载数据
data = load_boston()
target = data.target
data = data.data
# print(data.shape)

#数据标准化
# ss = StandardScaler
# X_2 = ss(data).copy
X_ = data
X_ = (X_ - np.mean(X_, axis=0)) / np.std(X_, axis=0)


# 目标值维度转换
y_ = target.reshape(target.shape[0], 1)
# print(y_.shape)
x_train, x_test, y_train, y_test = train_test_split(X_, y_, test_size = 0.25, random_state = 33)

#定义输入变量
X = x_train
y = y_train

import numpy as np
#定义维度
n, d_in, d_hidden, d_out = x_train.shape[0], x_train.shape[1], 16, 1
def linear(X, W, b):
    result = X.dot(W) + b
    return result

def Relu(y):
    return np.where(y < 0, 0, y)

def loss(y, y_pre):
    return np.mean(np.square(y - y_pre))
    
#构建两层神经网络 W1，b1  W2，b2  Relu 
def init_net():
    #初始化神经网络
    np.random.seed(33)
    W1 = np.random.randn(d_in, d_hidden)
    b1 = np.random.randn(d_hidden)
    W2 = np.random.randn(d_hidden, d_out)
    b2 = np.random.randn(d_out)
    return W1,W2,b1,b2
    
def forward(X, W1, W2, b1, b2):
    #线性层
    y1 = linear(X, W1, b1)
    ReLU = Relu(y1)
    y2 = linear(ReLU, W2, b2)
    return y1, ReLU, y2

#定义神经网络，返回值为权重
def NPnet(X, y, learning_rate = 1e-5, iter_num = 5000):
    W1, W2, b1, b2 = init_net()
    loss_temp = 0.0
    for i in range(iter_num):
        y1, ReLU, y_pre = forward(X, W1, W2, b1, b2)
        mse_loss = loss(y, y_pre)
        #print(mse_loss)
        #print(loss_temp)
        #早停法
        if (i > 0 and (np.abs(mse_loss - loss_temp) < 1e-10)) or (i == iter_num - 1):
            #print(mse_loss - loss_temp < 1e-10)
            return W1, W2, b1, b2, y_pre, i
        
        loss_temp = mse_loss.copy()
#         print(np.mean(y_pre - y))
        #输出层梯度
        grad_y_pre = 2.0 * (y_pre - y)
        #print(grad_y_pre.shape)
        #print(grad_y_pre)
        
        # W2 梯度更新
        grad_W2 = ReLU.T.dot(grad_y_pre)
        grad_b2 = grad_y_pre.sum(axis = 0)
        
        # 第二层(W2) 传递到的 Loss 值
        loss_W2 = grad_y_pre.dot(W2.T)
        #print(grad_Relu.shape)
        #print(y1.shape)
        #print(grad_W2.shape)
        #print(W2.shape)

        # ReLU 层梯度更新
        loss_ReLU = loss_W2.copy()
        loss_ReLU[y1 < 0] = 0
        
        #print(loss_ReLU)
        #print(loss_W2)
        
        # W1 更新权重
        #print(X)
        grad_W1 = X.T.dot(loss_ReLU)
        #print(grad_W1)
        grad_b1 = loss_ReLU.sum(axis = 0)
        #print(grad_b1.shape)
        W1 -= learning_rate * grad_W1
        
        # 更新 W2 
        W2 -= learning_rate * grad_W2
        b2 -= learning_rate * grad_b2
        #print(W2)
        b1 -= learning_rate * grad_b1
        #print(W1)
        

W1, W2, b1, b2, y_hat, i = NPnet(X, y)
print("W1:\n", W1)
print("W2:\n", W2)
# print(y)
# print(y_pre.reshape(1,y_pre.shape[0]))
# print(grad_y_pre)
# print(i)
# print(loss(y, y_hat))

y1, ReLU, y_pre = forward(x_test, W1, W2, b1, b2)
# print(y_pre, y_pre.shape)
print("MSE = {}".format(loss(y_test, y_pre)))

W1:
 [[ 4.60356869e-01 -1.53199023e+00 -1.59860248e+00 -5.24377603e-01
   1.56953777e-01  1.17024836e+00 -2.57409734e-01  1.04251780e+00
  -3.57124895e-01  3.84586059e-01 -5.27868979e-01 -8.16054582e-01
  -6.42512810e-01  3.64150817e-01 -1.17206429e+00 -2.06433198e+00]
 [-1.07408569e+00  1.12234293e+00 -8.23750680e-01  5.49637220e-01
   1.13567292e+00  7.44743607e-01 -2.22452963e+00 -2.24000088e-01
  -6.37504574e-01 -4.54058795e-01 -7.33122863e-01  3.41927116e-01
   8.91042451e-01 -7.36156525e-01 -1.23111841e+00 -8.63705200e-01]
 [ 1.27801219e+00  3.84091231e-01 -8.33583288e-01  5.30926906e-01
  -3.48377188e-01 -4.74753000e-01 -6.29172584e-01  9.14809573e-02
  -2.58940048e-01  7.05577955e-01  3.86551121e-01 -4.73106507e-01
  -9.40843835e-01  2.58882823e-01 -1.07338185e+00 -4.26696606e-01]
 [-2.51740430e-01  1.27160351e+00 -8.94590546e-01 -1.91210710e+00
   1.56786245e+00  2.18709786e-01 -5.13801092e-01  5.55945420e-01
  -1.01576823e+00  7.48400869e-01  2.15906380e+00 -8.36850662e-01
  

# 使用Numpy 构造的神经网络学习率太大时容易溢出，且梯度下降法不如 Adam 的优化方式，不容易调试

# 二、使用 pytorch

In [45]:
import pytorch

ModuleNotFoundError: No module named 'pytorch'

#  Action 2: 对移动推荐系统进行可视化数据探索
>数据集https://tianchi.aliyun.com/competition/entrance/231522/information\ntianchi_fresh_comp_train_item.csv\ntianchi_fresh_comp_train_user.csv  
比如时间规律统计，4种行为类别的对比…  

# 本章任务

In [1]:
import xlrd
data = xlrd.open_workbook('L7自测文档.xls')
#通过索引顺序获取
table = data.sheet_by_index(0)

""" 工作表中行/列的操作 """
#获取该sheet中的有效行数
nrows = table.nrows  
print(nrows)
row_index, col_index = 0, 0
# 获取某行信息
for row_index in range(2, nrows-6):
    print(table.row(row_index)[:2])
for row_index in range(nrows-5, nrows):
    print(table.row(row_index)[:2], table.row(row_index)[-2])

22
[text:'原理', text:'神经网络结构']
[text:'原理', text:'激活函数']
[text:'原理', text:'损失函数']
[text:'原理', text:'反向传播']
[text:'原理', text:'梯度下降']
[text:'原理', text:'优化方法（SGD、Adam）']
[text:'工具', text:'使用numpy搭建神经网络']
[text:'工具', text:'使用pytorch搭建神经网络']
[text:'原理', text:'Project：移动推荐系统']
[text:'原理', text:'探索性数据分析']
[text:'原理', text:'灵活使用dataframe']
[text:'原理', text:'时间函数使用']
[text:'工具', text:'分块读取海量数据']
[text:'工具', text:'特征工程']
[text:'Thinking1', text:'什么是反向传播中的链式法则'] text:'简要说明反向传播中的链式法则（10points）'
[text:'Thinking1', text:'请列举几种常见的激活函数，激活函数有什么作用'] text:'简要说明常用的激活函数及作用（10points）'
[text:'Thinking2', text:'利用梯度下降法训练神经网络，发现模型loss不变，可能有哪些问题？怎么解决？'] text:'能简要说明loss不变的解决方案（10points）'
[text:'Action1', text:'使用Pytorch编写神经网络，完成boston房价预测问题\n1）数据加载：from sklearn.datasets import load_boston\n2）网络结构：\nl1 = Linear(X, W1, b1)\ns1 = Relu(l1)\nl2 = Linear(s1, W2, b2)\ncost = MSE(y, l2)\n隐藏层维度为10\n'] text:'1、完成代码（20points）\n2、结果正确（10points）\n'
[text:'Action2', text:'对移动推荐系统进行可视化数据探索\n数据集https://tianchi.aliyun.com/competit